In [ ]:
!pip install pyspark

In [29]:
import json
import pandas as pd
import numpy as np
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from collections import Counter, defaultdict

import os
from os import listdir
from os.path import isfile, join
import pickle
import datetime
import hashlib
import math
import matplotlib
%matplotlib inline

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:
content_fp = "home/lemeiz/content_refine"
event_data = "one_week"

In [3]:
first_day = "20170101"
data = open("one_week/20170101.json")

In [ ]:
json_data = [json.loads(line) for line in data]

In [ ]:
list(json_data[3]['profile'])

In [6]:
article_json_files = [join(content_fp, f) for f in listdir(content_fp) if isfile(join(content_fp, f))]

In [22]:
x = json.load(open(article_json_files[1]))

In [36]:
location_dic = defaultdict(int)

for a in article_json_files:
    x = json.load(open(a))
    for idx, d in enumerate(x['fields']):
        if d['field']=='kw-location':
            if isinstance(d['value'], list):
                for loc in d['value']:
                    location_dic[loc] += 1
            elif isinstance(d['value'], str):
                location_dic[d['value']] += 1
            else:
                print(f"ERROR: {location_dic[d['value']]} was neither list nor string, it was {type(location_dic[d['value']])}")
                
filtered_dict = {k:v for k, v in location_dic if v < 10}

ValueError: too many values to unpack (expected 2)

In [44]:
filtered_dict = {k:v for k, v in location_dic.items() if v > 20}

In [45]:
sorted_filtered_dict = sorted(filtered_dict, key=filtered_dict.get)

In [48]:
sorted_dict = sorted(location_dic.items(), key=lambda x: x[1], reverse=True)

In [133]:
filtered_dict

{'frøya': 593,
 'trøndelag': 6795,
 'skottland': 168,
 'trondheim': 21922,
 'oslo': 7393,
 'new york': 816,
 'milano': 73,
 'sør-trøndelag': 6671,
 'las palmas': 40,
 'afrika': 304,
 'india': 265,
 'gran canaria': 182,
 'mexico': 202,
 'sri lanka': 47,
 'canada': 449,
 'norge': 13560,
 'nord-norge': 517,
 'london': 1148,
 'tromsø': 853,
 'otta': 28,
 'hellas': 320,
 'verdal': 1131,
 'afghanistan': 347,
 'russland': 932,
 'ukraina': 171,
 'europa': 2629,
 'stiklestad': 259,
 'lahti': 463,
 'danmark': 1138,
 'røros': 1039,
 'norden': 455,
 'molde': 658,
 'nord-trøndelag': 3017,
 'grong': 239,
 'høylandet': 130,
 'kristiansund': 529,
 'asker': 210,
 'melhus': 1159,
 'midt-norge': 2333,
 'grønland': 120,
 'bergen': 2308,
 'indonesia': 90,
 'kina': 574,
 'oppdal': 1109,
 'stockholm': 612,
 'berg': 158,
 'oliver': 37,
 'luna': 25,
 'selbu': 539,
 'åfjord': 340,
 'steinkjer': 1343,
 'hemne': 205,
 'midtre gauldal': 262,
 'charlottenlund': 294,
 'malvik': 840,
 'sverige': 2378,
 'ørland': 509,

In [51]:
postal_codes = json.load(open('postal-codes.json'))

In [142]:
poststeder = []
kommuner = []

for x in postal_codes.values():
    if x[0].lower() not in poststeder:
        poststeder.append(x[0].lower())
        
    if x[2].lower() not in kommuner:
        kommuner.append(x[2].lower())


In [143]:
acc = []
komacc = []
unacc = []
for stedsnavn in filtered_dict.keys():
    if stedsnavn.lower() in poststeder:
        acc.append(stedsnavn.lower())
    elif stedsnavn.lower() in kommuner:
        komacc.append(stedsnavn.lower())
    else:
        unacc.append(stedsnavn.lower())

In [146]:
valid = []
kom_val = []
invalid = []
no_loc = []

for art_idx, a in enumerate(article_json_files):
    art_val = 0
    loc_found = False
    x = json.load(open(a))
    for idx, d in enumerate(x['fields']):
        if d['field']=='kw-location':
            loc_found = True
            if isinstance(d['value'], list):
                for loc in d['value']:
                    if loc.lower() in poststeder:
                        valid.append(art_idx)
                        art_val = 1
                        break
                if not art_val:
                    for loc in d['value']:
                        if loc.lower() in kommuner:
                            kom_val.append(art_idx)
                            art_val = 1
                            break
            elif isinstance(d['value'], str):
                if d['value'].lower() in poststeder:
                    valid.append(art_idx)
                    art_val = 1
                elif d['value'].lower() in kommuner:
                    kom_val.append(art_idx)
                    art_val = 1
            else:
                print(f"ERROR: {location_dic[d['value']]} was neither list nor string, it was {type(location_dic[d['value']])}")
            if not art_val:
                invalid.append(art_idx)
            break
            
    if not loc_found:
        no_loc.append(art_idx)


In [150]:
(len(valid)+len(kom_val)+len(invalid)+len(no_loc)) == len(article_json_files)

True

In [129]:
json.load(open(article_json_files[231]))

{'fields': [{'field': 'modifiedtime',
   'type': 'time',
   'value': '2017-01-24T20:29:49.000Z'},
  {'field': 'adressa-access', 'value': 'subscriber'},
  {'field': 'adressa-importance', 'value': '0'},
  {'field': 'adressa-tag',
   'value': ['Signert',
    'debatt',
    'Vindkraft',
    'klima',
    'energi',
    'Kampflybasen',
    'Fosen',
    'Selbu',
    'Meråker',
    'Snillfjord',
    'stein arne Sæther',
    'Naturvernforbundet']},
  {'field': 'author', 'value': ['iwar arnstad', 'svensk trønder.']},
  {'field': 'body',
   'value': ['Saken oppdateres.',
    'Mange i Trøndelag er sterkt imot satsing på vindkraft. Ikke jeg.',
    'Vi vet at fossile brensler truer vårt klima og det biologiske mangfoldet. Ingen kan heller si hvordan atomavfallet skal forvares på en sikker måte. Ikke slik at det i 10 000-talls år, så lenge det er livsfarlig radioaktivt, kan utelukkes at nåværende eller framtidige terrorister, gale statsledere eller naturkatastrofer vil kunne forårsake utslipp.',
    'D

In [113]:
no_loc_no_caveat = []
for art_idx in no_loc:
    art_val = 0
    body_found = False
    x = json.load(open(article_json_files[art_idx]))
    for d in x['fields']:
        if d['field']=='category':
            body_found = True
            if d['value'][0].lower() != 'saken oppdateres.':
                no_loc_no_caveat.append([art_idx, d['value'][0]])
            break
    if not body_found:
        print(f"No body found in article index {art_idx}")
            
    

No body found in articcle index 221
No body found in articcle index 236
No body found in articcle index 282
No body found in articcle index 478
No body found in articcle index 523
No body found in articcle index 608
No body found in articcle index 676
No body found in articcle index 958
No body found in articcle index 969
No body found in articcle index 993
No body found in articcle index 1025
No body found in articcle index 1096
No body found in articcle index 1177
No body found in articcle index 1412
No body found in articcle index 1442
No body found in articcle index 1526
No body found in articcle index 1664
No body found in articcle index 1681
No body found in articcle index 1773
No body found in articcle index 2042
No body found in articcle index 2430
No body found in articcle index 2444
No body found in articcle index 2513
No body found in articcle index 2515
No body found in articcle index 2535
No body found in articcle index 2651
No body found in articcle index 2656
No body fou

No body found in articcle index 23217
No body found in articcle index 23282
No body found in articcle index 23350
No body found in articcle index 23565
No body found in articcle index 23586
No body found in articcle index 23698
No body found in articcle index 23748
No body found in articcle index 23917
No body found in articcle index 23975
No body found in articcle index 24006
No body found in articcle index 24127
No body found in articcle index 24142
No body found in articcle index 24145
No body found in articcle index 24242
No body found in articcle index 24367
No body found in articcle index 24573
No body found in articcle index 24679
No body found in articcle index 25044
No body found in articcle index 25075
No body found in articcle index 25099
No body found in articcle index 25150
No body found in articcle index 25165
No body found in articcle index 25175
No body found in articcle index 25337
No body found in articcle index 25366
No body found in articcle index 25403
No body foun

No body found in articcle index 44204
No body found in articcle index 44321
No body found in articcle index 44323
No body found in articcle index 44358
No body found in articcle index 44400
No body found in articcle index 44408
No body found in articcle index 44592
No body found in articcle index 44636
No body found in articcle index 44671
No body found in articcle index 44676
No body found in articcle index 44689
No body found in articcle index 44946
No body found in articcle index 44957
No body found in articcle index 45091
No body found in articcle index 45103
No body found in articcle index 45124
No body found in articcle index 45218
No body found in articcle index 45430
No body found in articcle index 45475
No body found in articcle index 45642
No body found in articcle index 45704
No body found in articcle index 45809
No body found in articcle index 46154
No body found in articcle index 46156
No body found in articcle index 46183
No body found in articcle index 46189
No body foun

No body found in articcle index 67733
No body found in articcle index 67761
No body found in articcle index 67822
No body found in articcle index 68000
No body found in articcle index 68346
No body found in articcle index 68514
No body found in articcle index 68515
No body found in articcle index 68740
No body found in articcle index 68751
No body found in articcle index 68810
No body found in articcle index 68865
No body found in articcle index 68945
No body found in articcle index 69115
No body found in articcle index 69205
No body found in articcle index 69208
No body found in articcle index 69377
No body found in articcle index 69497
No body found in articcle index 69504
No body found in articcle index 69574
No body found in articcle index 69642
No body found in articcle index 69669
No body found in articcle index 69694
No body found in articcle index 69762
No body found in articcle index 69910
No body found in articcle index 69935
No body found in articcle index 69957
No body foun

In [120]:
len(no_loc)/len(article_json_files)

0.17836444729321901

In [134]:
CATEGORIES_TO_IGNORE = ['bolig', 'abonnement']
SITES_TO_IGNORE = ['kundeservice.adressa.no']

In [140]:
abbo_art = []
for idx, art in enumerate(article_json_files):
    art_val = 0
    abbo_found = False
    x = json.load(open(art))
    for d in x['fields']:
        if d['field']=='kw-category':
            body_found = True
            if any(x in CATEGORIES_TO_IGNORE for x in d['value']):
                abbo_art.append(idx)
            break
    if not body_found:
        print(f"No Category found in article index {idx}")

In [141]:
len(abbo_art)

784

In [139]:
len()

{'fields': [{'field': 'modifiedtime',
   'type': 'time',
   'value': '2012-11-07T12:33:13.000Z'},
  {'field': 'author', 'value': 'kundeservice administrator'},
  {'field': 'body',
   'value': ['Registrering Vi har ulike tjenester som krever at du registrerer informasjon om deg selv,\nslik som e-postadresse, navn, kjønn, alder og interesser. Disse dataene lagres\ni databaser internt i Adresseavisen og brukes ikke til andre formål enn til å\nyte kundeservice og administrere ditt kundeforhold. Enhver bruker står fritt\ntil å be om å få fjernet de opplysningene han har lagt inn i disse basene.',
    'Statistikk Ved å ta i bruk applikasjonen samtykker du til at det samles inn statistikk på\naggregert nivå om hvordan applikasjonen brukes. Eksempler på slik statistikk er\nantall brukere, antall visninger av artikler, navigasjonsmønster etc.\nStatistikken vil aldri inneholde noen form for personlig informasjon, og vil\nikke kunne kobles til din brukerkonto hos Adresseavisen, dersom du har dett

In [151]:
kommuner

['oslo',
 'bærum',
 'asker',
 'nordre follo',
 'ås',
 'frogn',
 'nesodden',
 'lørenskog',
 'lillestrøm',
 'nittedal',
 'moss',
 'vestby',
 'våler (viken)',
 'fredrikstad',
 'råde',
 'hvaler',
 'sarpsborg',
 'halden',
 'aremark',
 'indre østfold',
 'skiptvet',
 'marker',
 'rakkestad',
 'enebakk',
 'nes',
 'aurskog-høland',
 'rælingen',
 'gjerdrum',
 'nannestad',
 'ullensaker',
 'eidsvoll',
 'hurdal',
 'sør-odal',
 'nord-odal',
 'kongsvinger',
 'eidskog',
 'grue',
 'åsnes',
 'hamar',
 'stange',
 'ringsaker',
 'løten',
 'elverum',
 'trysil',
 'våler (innlandet)',
 'engerdal',
 'åmot',
 'stor-elvdal',
 'rendalen',
 'tynset',
 'tolga',
 'os',
 'alvdal',
 'folldal',
 'lillehammer',
 'ringebu',
 'øyer',
 'nord-fron',
 'sør-fron',
 'gausdal',
 'dovre',
 'lesja',
 'sel',
 'vågå',
 'lom',
 'skjåk',
 'gran',
 'lunner',
 'gjøvik',
 'østre toten',
 'vestre toten',
 'søndre land',
 'nordre land',
 'etnedal',
 'nord-aurdal',
 'sør-aurdal',
 'øystre slidre',
 'vestre slidre',
 'vang',
 'drammen',
 'ho